In [1]:
# Install gradio
%%capture
!pip install gradio

In [2]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import everything that is needed
import gradio as gr
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.nn.functional import softmax

In [7]:
# Load models and tokenizer
base_model = GPT2LMHeadModel.from_pretrained("gpt2")
anti_expert_model = GPT2LMHeadModel.from_pretrained("distilgpt2")
expert_model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Colab_Notebooks/Transformers/fine_tuned_model")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Move models to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)
anti_expert_model.to(device)
expert_model.to(device)

# Proxy-Tuning Function
def generate_answers(question):
    # Tokenize input
    inputs = tokenizer(question, return_tensors="pt").to(device)

    # Compute logits for all models
    with torch.no_grad():
        base_logits = base_model(**inputs).logits
        anti_expert_logits = anti_expert_model(**inputs).logits
        expert_logits = expert_model(**inputs).logits

    # Proxy-tuned logits
    alpha = 2.0  # Adjust to see what works best
    proxy_tuned_logits = base_logits + alpha * (expert_logits - anti_expert_logits)

    # Generate GPT-2 answer
    base_answer_ids = torch.argmax(softmax(base_logits, dim=-1), dim=-1)
    gpt2_answer = tokenizer.decode(base_answer_ids[0], skip_special_tokens=True)

    # Generate Proxy-Tuned answer
    proxy_tuned_answer_ids = torch.argmax(softmax(proxy_tuned_logits, dim=-1), dim=-1)
    proxy_tuned_answer = tokenizer.decode(proxy_tuned_answer_ids[0], skip_special_tokens=True)

    return gpt2_answer, proxy_tuned_answer

# Gradio Interface
def process_input(question):
    gpt2_answer, proxy_tuned_answer = generate_answers(question)
    return f"GPT-2 Answer: {gpt2_answer}\nProxy-Tuned Answer: {proxy_tuned_answer}"

# Set up Gradio app
interface = gr.Interface(
    fn=process_input,
    inputs="text",
    outputs="text",
    title="Proxy-Tuning Demonstration",
    description="Enter a question and see answers generated by GPT-2 and Proxy-Tuned GPT-2."
)

# Launch Gradio interface
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://075c90ca4e06505f1e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
